# Set-up SQLite Database

In [30]:
import sqlite3
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.36.0


In [31]:
product = pd.read_csv('../../data/product.csv')
product_price = pd.read_csv('../../data/product_price.csv')
vendor = pd.read_csv('../../data/vendor.csv')
store = pd.read_csv('../../data/store.csv')
transaction = pd.read_csv('../../data/transaction.csv')
county_table = pd.read_csv('../../data/county.csv')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Last Edits to Tables

In [36]:
store.rename(columns = {'Store Number':'StoreID',
                        'Store Name':'StoreName',
                        'Zip Code':'ZipCode',
                       'avg order interval':'avg_order_interval',
                       'Density 5mi':'Density_5mi'}, inplace = True)
store['StoreName'] = store['StoreName'].str.lower()

# Create store type
store['StoreType'] = 'Unknown'
store.loc[store.StoreName.str.contains('liquor|spirits|private cellar, inc|wine|cork \
|kimberly mart|the secret cellar|kick the can|mccoy|franklin street floral & gift|benz distributing'), 'StoreType'] = 'Liquor, Spirits & Wine'
store.loc[store.StoreName.str.contains('beer city|the cooler|bani|booze|cornerstone apothecary'), 'StoreType'] = 'Liquor, Spirits & Wine'
store.loc[store.StoreName.str.contains("smoker's genie"), 'StoreType'] = 'Liquor, Spirits & Wine'
store.loc[store.StoreName.str.contains('beer thirty|g corner|blind pig cocktail bodega|beer on floyd'), 'StoreType'] = 'Liquor, Spirits & Wine'
store.loc[store.StoreName.str.contains('great pastimes|sodes green acre|kc brothers|leo1'), 'StoreType'] = 'Liquor, Spirits & Wine'
store.loc[store.StoreName.str.contains("ali's corner"), 'StoreType'] = 'Liquor, Spirits & Wine'
store.loc[store.StoreName.str.contains("central city 2"), 'StoreType'] = 'Liquor, Spirits & Wine'

store.loc[store.StoreName.str.contains('bar|last call 2|catfish charlie|eichman enterprises inc'), 'StoreType'] = 'Distillery/Bar/Restaurant'
store.loc[store.StoreName.str.contains('distillery|distilery|distillation|distilling| \
distillers|cedar ridge vineyards'), 'StoreType'] = 'Distillery/Bar/Restaurant'

store.loc[store.StoreName.str.contains('drugstore|drug|pharmacy|walgreens'), 'StoreType'] = 'Drugstore'

store.loc[store.StoreName.str.contains('convenience|honk|beverage|quick|gas|quik|pit stop| \
econ-o-mart|shop n save|kwik|cigarette|tobacco|smoke shop|fast mart|b and b west|mini mart| \
minimart|kellogg country store|prime mart|karam kaur khasriya llc|five star snacks and more'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('best trip|econ-o-mart|oky doky|country stop'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('expo mart|bucky|kum & go|circle k|car-go-express'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('ez stop|speede shop|petro mart|one stop'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('select mart|b p|the pumper|indy 66|jiffy express|dyno'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('fast break|jiffy mart|truckstop|stop|ez mart|bp'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('metro mart|travel center|xpress|conoco|shell'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('river mart|decorah mart|music station|river mart|the station'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('central mart|country store|cash saver|phillips'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('on the go|circle s|c-store|road ranger|mart|yesway'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('new star|corner store|fasttrak|the boonedocks|cubby|the depot'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('mega saver|pump|git-n-go|oasis|easygo|filling station'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('snack shack|casey|mod|hop n shop|brew oil'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains("stan's corner"), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('raceway|boji junction|pronto|thunder ridge ampride'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains("todd's"), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('l&m mighty shop|oskaloosa watering hole'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains("raymond's station"), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('petro|oak street station llc|williams boulevard service'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('avenue g store|hampton crossroads|day break|sinclair'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('lincoln farm and home|d&t|the hut|fuel|heartland|depot norway'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('mcdermott oil co|kimmes wall lake|prime star'), 'StoreType'] = 'Convenience'
store.loc[store.StoreName.str.contains('crossroads of algona|jumpstart'), 'StoreType'] = 'Convenience'

store.loc[store.StoreName.str.contains('target|wal-mart|sam|costco'), 'StoreType'] = 'Dept. Store'

store.loc[store.StoreName.str.contains('food|supermarket|grocery|market|dahl|osco|super valu| \
pantry|huber|hy-vee|save a lot|super saver|thriftway|fareway|wapello jack and jill'), 'StoreType'] = 'Food & Grocery'
store.loc[store.StoreName.str.contains('groceries|price chopper|jack n jill'), 'StoreType'] = 'Food & Grocery'
store.loc[store.StoreName.str.contains('jack & jill|save more|jim and charlies affiliated'), 'StoreType'] = 'Food & Grocery'
store.loc[store.StoreName.str.contains('lansing iga|corwith farm service'), 'StoreType'] = 'Food & Grocery'
store.loc[store.StoreName.str.contains('general store|dollar store'), 'StoreType'] = 'Food & Grocery'

store.loc[store.StoreName.str.contains('adventureland inn|marriott|prairie meadows|casino|10th hole inn & suite'), 'StoreType'] = 'Hotel/Casino'
store.loc[store.StoreName.str.contains('honey creek resort state park'), 'StoreType'] = 'Hotel/Casino'

In [37]:
store.groupby('StoreType').size()

StoreType
Convenience                   888
Dept. Store                    96
Distillery/Bar/Restaurant      41
Drugstore                      96
Food & Grocery               1211
Hotel/Casino                   12
Liquor, Spirits & Wine        253
Unknown                        93
dtype: int64

In [20]:
#store.to_csv('../../data/store.csv', index_label = False)

In [38]:
# remove these columns
product = product.drop(['CategoryCode','CategoryName'], axis=1)

In [39]:
# rename to remove space
county_table.rename(columns = {'UNEMPLOYMENT RATE':'UNEMPLOYMENT_RATE'}, inplace = True)

In [40]:
print('product: ', product.columns)
print('product_price: ', product_price.columns)
print('vendor: ', vendor.columns)
print('store: ', store.columns)
print('transaction: ', transaction.columns)
print('county: ', county_table.columns)

product:  Index(['ProductID', 'ProductDescription', 'CategoryGroup', 'Proof',
       'VendorID'],
      dtype='object')
product_price:  Index(['ProductID', 'Date', 'StateBottleRetail', 'StateBottleCost'], dtype='object')
vendor:  Index(['VendorID', 'VendorName'], dtype='object')
store:  Index(['StoreID', 'StoreName', 'County', 'lat', 'lng', 'max', 'min', 'Freq',
       'avg_order_interval', 'active', 'Density_5mi', 'StoreType'],
      dtype='object')
transaction:  Index(['TransactionID', 'Date', 'Year', 'StoreID', 'ProductID', 'VendorID',
       'Pack', 'BottleVolume_ml', 'BottlesSold', 'VolumeSold_Liters',
       'Sale_Dollars'],
      dtype='object')
county:  Index(['County', 'Year', 'Population', 'Income_PerCapita', 'LABORFORCE',
       'EMPLOYMENT', 'UNEMPLOYMENT', 'UNEMPLOYMENT_RATE'],
      dtype='object')


# Create Database

In [41]:
# delete database (in folder) if you have added a table previously 
dbconn = sqlite3.connect('../../data/IowaLiquorSales.db') # database
cursor = dbconn.cursor() # create a cursor to use to execute sql statements

In [42]:
cursor.execute('''CREATE TABLE Product
                ([ProductID] TEXT PRIMARY KEY, 
                [ProductDescription] text, 
                [CategoryGroup] text, 
                [Proof] real, 
                [VendorID] integer)''')

cursor.execute('''CREATE TABLE Price
                ([ProductID] text, 
                [Date] date, 
                [StateBottleRetail] real, 
                [StateBottleCost] real,
                PRIMARY KEY(ProductID, Date, StateBottleRetail))''')

cursor.execute('''CREATE TABLE Vendor
                ([VendorID] INTEGER PRIMARY KEY, 
                [VendorName] text)''')

cursor.execute('''CREATE TABLE Store
                ([StoreID] INTEGER PRIMARY KEY, 
                [StoreName] text, 
                [Address] text, 
                [City] text, 
                [ZipCode] text, 
                [County] text, 
                [lat] real, 
                [lng] real, 
                [max] date, 
                [min] date,
                [Freq] real,
                [avg_order_interval] integer,
                [active] text,
                [Density_5mi] integer,
                [StoreType] text)''')

cursor.execute('''CREATE TABLE Sales
                ([TransactionID]  TEXT PRIMARY KEY, 
                [Date] date, 
                [Year] text,
                [StoreID] integer, 
                [ProductID] text, 
                [VendorID] integer, 
                [Pack] integer,
                [BottleVolume_ml] real,
                [BottlesSold] integer, 
                [VolumeSold_Liters] real, 
                [Sale_Dollars] real)''')

cursor.execute('''CREATE TABLE County      
                ([County] text, 
                [Year] text, 
                [Population] integer, 
                [Income_PerCapita] integer, 
                [LABORFORCE] integer, 
                [EMPLOYMENT] integer,
                [UNEMPLOYMENT] real,
                [UNEMPLOYMENT_RATE] real,
                PRIMARY KEY(County, Year))''')

In [43]:
product.to_sql('Product', dbconn, if_exists = 'append', index = False)
product_price.to_sql('Price', dbconn, if_exists = 'append', index = False)
vendor.to_sql('Vendor', dbconn, if_exists = 'append', index = False)
store.to_sql('Store', dbconn, if_exists = 'append', index = False)
transaction.to_sql('Sales', dbconn, if_exists = 'append', index = False)
county_table.to_sql('County', dbconn, if_exists = 'append', index = False)

In [44]:
dbconn.commit()

In [45]:
dbconn.close()